Check the new T2 TNS selector, then try to get the client to work with all the config updates..

In [ ]:
import os
%load_ext ampel_quick_import
%qi DevAmpelContext DictSecretProvider AmpelLogger T2Processor ChannelModel AlertConsumer TarAlertLoader ChannelModel AbsAlertFilter T3Processor TransientInfoPrinter

In [ ]:
# See other sample notebooks for path to download
ALERT_ARCHIVE = "/home/jnordin/data/ztfalerts/ztf_public_20180731_nostars.tar.gz"
AMPEL_CONF = "/home/jnordin/github/ampelv71/ampel_conf.yaml"

In [ ]:
secrets = DictSecretProvider( {'slack/ztf_general/jno':None,
                              'mongo/logger':{},
                              'mongo/writer':{},
                               'fritz/jvs/ampelbot': None,
                               'desi/skyportal/jno': None
                               'tns/jnordin': None,
                               'extcats/reader' : {},
                               'lco/jnordin':None,
                              })

In [ ]:
ctx = DevAmpelContext.load(
    config_file_path = AMPEL_CONF,
    db_prefix = "TNSTest",
    purge_db = True,
    secrets = secrets,
)

In [ ]:
# First channel, general look combined with SNGuess
ctx.add_channel(
    name="TNS1",
    access=['ZTF', 'ZTF_PUB']
)

In [ ]:
cat_conf = { 'catalogs' : {
             'SDSS_spec' : {
                 'use' : 'extcats',
                 'catq_kwargs' : {
                     'ra_key' : 'ra',
                     'dec_key' : 'dec',
                 },
                 'rs_arcsec' : 10,
                 'keys_to_append' : ['z','bptclass','subclass'],
                 'pre_filter' : None,
                 'post_filter' : None,
             },
             'milliquas' : {
                 'use' : 'extcats',
                 'catq_kwargs' : {
                     'ra_key' : 'ra',
                     'dec_key' : 'dec',
                 },
                 'rs_arcsec' : 3,
                 'keys_to_append' : ['broad_type', 'name', 'redshift', 'qso_prob'],
                 'pre_filter' : None,
                 'post_filter' : None,
             }
        },
        'extcats' : {
            'auth' : {'key' : 'extcats/reader'},
            'max_time' : 300.0            
        }
}

In [ ]:
extcat_config_id = ctx.add_config_id( cat_conf )
print(extcat_config_id)

In [ ]:
tnseval_conf = {
    "t2_dependency": [
        {
            "unit": "T2CatalogMatch",
            "config": extcat_config_id,
            "link_override": {
                "pps": "last"
                }
        }
    ]
}

In [ ]:
tnseval_config_id = ctx.add_config_id( tnseval_conf )
print(tnseval_config_id)

In [ ]:
ap = AlertConsumer(
    context = ctx,
    process_name = "ipyton_notebook_test",
    supplier = "ZiAlertSupplier",
    log_profile = "debug",
    directives = [
        {
            "channel": "TNS1", 
            "filter": {"unit": "SimpleDecentFilter","config": {
#                                                                "min_drb":0.995,
                                                                "min_rb":0.3,
                                                                "min_ndet":3,
                                                                "min_tspan":0.02,
                                                                "max_tspan" : 999,
                                                                "min_gal_lat" : 14
                                                              }
                        },
            "stock_update": "ZiStockIngester",
            't0_add': {
                "ingester": "ZiAlertContentIngester",
                "t1_combine": [
                    {
                        "ingester": "PhotoCompoundIngester",
                        "config": {"combiner": "ZiT1Combiner"},
                        "t2_compute": {
                            "ingester": "PhotoT2Ingester",
                            "config": {"tags": ["ZTF"]},
                            "units": [
                                {'unit': 'T2TNSEval',
                                 'config': tnseval_config_id
                                },
                                {'unit': 'T2BrightSNProb'
                                },
                            ]
                        }
                    }
                ],
                "t2_compute": {
                    'unit': 'DualPointT2Ingester',
                    'config': {'tags': ['ZTF']},
                    'units': [
                        {
                            'unit': 'T2CatalogMatch', 
                            'config' : extcat_config_id,
                            "ingest": {'eligible': {'pps': 'first'}}
                        }
                    ],
                },

            }
        }
    ]
)

In [ ]:
ap.set_loader(TarAlertLoader(ALERT_ARCHIVE))

In [ ]:
ap.set_iter_max(20)

In [ ]:
ap.run()

In [ ]:
t2p = T2Processor(context=ctx, process_name="T2Processor_test", log_profile="debug")

In [ ]:
t2p.run()

In [ ]:
t3p = T3Processor(
    context=ctx,
    process_name = "T3Processor_test",
    log_profile = "default", # debug
    channel = "TNS1",
    directives = [ {
        "context": [
            {"unit": "T3AddLastRunTime"},
            {"unit": "T3AddAlertsNumber"}
        ],
        "select": {
            "unit": "T3FilteringStockSelector",
            "config": {
                "created": {"after": {"match_type": "time_delta", "days": -400000}},
                'tag': {'with': "ZTF", 'without': 'HAS_ERROR'},
                't2_filter':  {
                    'unit': 'T2TNSEval',
                    'match': {'tns_candidate':True },
                }, 
            }
        },
        "load": {
            "unit": "T3SimpleDataLoader",
            "config": {
                "directives": ["TRANSIENT", "T2RECORD"],
            }

        },
        "run": {
            "unit": "T3UnitRunner",
            "config": {
                "directives": [
                      {
                            "project": {
                                "unit": "T3ChannelProjector",
                                "config": {
                                    "channel": "TNS1"
                                }
                            },
                            "execute": [
                                {"unit": "TransientInfoPrinter"},
                                {
                                    "unit": "TNSTalker",
                                    "config": {
                                        "tns_api_key": {
                                            "key":"tns/jnordin",
                                        },
                                        "sandbox":True,
                                        "submit_tns":True,
                                    },
                                },
                            ]

                      }
                ]
            }
        }
    } ]
)


In [ ]:
t3p.run()